In [1]:
!pip install faiss-cpu
!pip install mistralai

   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.7 MB 220.2 kB/s eta 0:01:03
   ---------------------------------------- 0.1/13.7 MB 297.7 kB/s eta 0:00:46
   ---------------------------------------- 0.1/13.7 MB 437.6 kB/s eta 0:00:32
    --------------------------------------- 0.2/13.7 MB 583.1 kB/s eta 0:00:24
    --------------------------------------- 0.2/13.7 MB 689.2 kB/s eta 0:00:20
   - -------------------------------------- 0.4/13.7 MB 1.0 MB/s eta 0:00:14
   - -------------------------------------- 0.5/13.7 MB 1.1 MB/s eta 0:00:13
   -- ------------------------------------- 0.7/13.7 MB 1.6 MB/s eta 0:00:09
   -- ------------------------------------- 1.0/13.7 MB 1.9 MB/s eta 0:00:07
   --- ----------

In [2]:
import os
os.environ["MISTRAL_API_KEY"] = "WqsWBSZBIqtk3keTGAp637ICEoOBffJH"
print(f"MISTRAL_API_KEY: {os.environ.get('MISTRAL_API_KEY')}")

MISTRAL_API_KEY: WqsWBSZBIqtk3keTGAp637ICEoOBffJH


In [3]:
api_key = os.getenv("MISTRAL_API_KEY")

In [42]:
import requests
from bs4 import BeautifulSoup
import re
response = requests.get(
"https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/sport-and-wellness-facilities-and")
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")
tag = soup.find("div")
text = tag.text
print(text)


















Search













Search by Keyword


Search








Search by Purpose

I am
Select an OptionUDST EmployeeCareer at UDSTParentCurrent StudentAlumniFuture Student



Looking for
Select an Option

Take me There



































Quick links



Current Student



Registration




Academic Calendar




Student Systems




Attendance




Final Exams




Academic Awards




Request Letters and Records




Graduation Ceremony




Graduation Requirements




Ramadan Schedule







Professional Education



Customized Training Courses




Consulting Services




Register Today




Online Resources







Alumni



Presidents Message




Alumni Services




Update Contact Information




Request Letters, Records and Transcripts







Alumni Benefits




UDST Offers







Alumni Events




Alumni Magazine




Alumni Office




Graduation Ceremonies







Testing Centre



Placement Tests




IELTS Tests




Proctoring







Careers




Sport and Wellness



In [43]:
file_name = "Text.txt"
with open(file_name, 'w') as file:
    file.write(text)

In [44]:
chunk_size = 512
chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

In [45]:
len(chunks)

62

In [46]:
import os
from mistralai import Mistral
def get_text_embedding(list_txt_chunks):
    client = Mistral(api_key=api_key)
    embeddings_batch_response = client.embeddings.create(model="mistral-embed",
                                                         inputs=list_txt_chunks)
    return embeddings_batch_response.data

In [47]:
import numpy as np
text_embeddings = get_text_embedding(chunks)

In [48]:
len(text_embeddings)

62

In [49]:
len(text_embeddings[0].embedding)

1024

In [50]:
embeddings = np.array([text_embeddings[i].embedding for i in range(len(text_embeddings))])

In [51]:
import faiss

In [52]:
d = len(text_embeddings[0].embedding)
index = faiss.IndexFlatL2(d)
index.add(embeddings)

In [53]:
question = "What are the way I can use sports facilities?"
question_embeddings = np.array([get_text_embedding([question])[0].embedding])

In [54]:
question_embeddings

array([[ 0.03643799,  0.04135132,  0.03222656, ..., -0.01273346,
         0.05230713, -0.00886536]])

In [55]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[59 56]]


In [56]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['ply.\n            \n\n\n\n                S&W Facility Reservation Capabilities\n            \n\n                Able to register for various S&W Services\n            \n\n                First Come, First Serve Basis.\n            \n\nSport and Wellness Extended Community Access Hours\nAD HOC Tier\nCollege Partners\n\n\n                Dependent on the nature of facilities and/or services requested, as per the written agreement concluded for this purpose.\n            \n\n\n\n                Access and Facility usage is subject t', "'s administrative staff.\n\n\n                Standard Facility Reservation Rates Apply.\n            \n\n                Service registration fees may apply, in which case rates will be discounted.\n            \n\n\n\n                S&W Facility Reservation Capabilities\n            \n\n                Able to register for various S&W Services\n            \n\n                First Come, First Serve Basis.\n            \n\nSport and Wellness Extended 

In [57]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [58]:
from mistralai import Mistral, UserMessage
def mistral(user_message, model="mistral-small-latest", is_json=False):
    model = "mistral-large-latest"
    client = Mistral(api_key=api_key)
    messages = [
        UserMessage(content=user_message),
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages,)
    return chat_response.choices[0].message.content

In [59]:
response = mistral(prompt)
print(response)

Based on the provided context, here are the ways you can use sports facilities:

1. **Reservation:** You can register for various Sports & Wellness (S&W) services and reserve facilities. This is done on a first come, first serve basis.

2. **Extended Community Access Hours:** There are extended access hours for different tiers of the community:
   - **AD HOC Tier:** For college partners, access depends on the nature of facilities and/or services requested, as per a written agreement.
   - **Tier 2:** For alumni, standard facility reservation rates apply.

3. **Fees:** Service registration fees may apply, but discounted rates might be available.

To use the facilities, you should contact the administrative staff as access and usage are subject to their oversight. Standard facility reservation rates will apply unless otherwise specified.
